In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import ANUBIS_triggered_functions as ANT
import pandas as pd
import matplotlib.backends.backend_pdf
from matplotlib.ticker import MultipleLocator
import plotly
import plotly.graph_objs as go
import plotly.express as px
import csv 


%matplotlib inline
current_directory=  os.path.dirname(os.getcwd())

In [31]:
def countChannels_Timed_UNCORRUPTED_byChannel(events):
    #Expects events from one TDC, counts how many hits each channel has within the event list

    chanCounts = [[] for x in range(128)]
    channel_corruptions = [0 for x in range(128)]
    number_corruptions = 0

    for event in events:
        for word in event:
            #get time of event
            time = word&0xfffff

            #ADD CHECK TO SEE THAT DATA IS NOT CORRUPTED 
            if 0 <= time <= 1250:
                try:
                    #Append time of event to specific channel triggered to get event.
                    chanCounts[(word>>24)&0x7f].append(time)
                except:
                    print(word>>24)
            else:
                number_corruptions+=1
                chanCounts[(word>>24)&0x7f].append("CORRUPT")
                channel_corruptions[(word>>24)&0x7f]+=1

    return chanCounts, number_corruptions, channel_corruptions

def divideHitCountsByRPC_Timed_UNCORRUPTED(data):
    #Divides the number of hits in each channel into individual RPCs
    etaHits = [[],[],[],[],[],[]]
    phiHits = [[],[],[],[],[],[]]

    etaCorruption = [np.zeros(32) for x in range(6)]
    phiCorruption = [np.zeros(64) for x in range(6)]

    #Store populations of corruption per event 

    number_corruptions= []
    tdc_corruptions = [0,0,0,0,0]

    for event in range(0,len(data[0])):

        event_corruptions = 0

        corrupted_channels = [ [] for x in range(5)]
        
        tdcCounts = []
        for tdc in range(5):
            chanCounts, n_corruptions, channel_corruptions = countChannels_Timed_UNCORRUPTED_byChannel([data[tdc][event]])
            if n_corruptions == 0:
                tdcCounts.append(chanCounts)
            else:
                event_corruptions += n_corruptions
                tdc_corruptions[tdc] += n_corruptions

            corrupted_channels[tdc]= channel_corruptions

        #ONLY APPEND EVENT IF IT DOES NOT INCLUDE CORRUPTED DATA
        
        if len(tdcCounts) == 5:
            #No corrupted events

            etaHits[0].append(tdcCounts[0][0:32]) #Triplet Eta Low
            phiHits[0].append(tdcCounts[0][32:96]) #Triplet Phi low
            etaHits[1].append(tdcCounts[0][96:128]) #Triplet Eta Mid
            phiHits[1].append(tdcCounts[1][0:64]) #Triplet Phi Mid
            etaHits[2].append(tdcCounts[1][64:96]) #Triplet Eta Top
            phiHits[2].append(tdcCounts[1][96:128]+tdcCounts[2][0:32]) #Triplet Phi Top
            etaHits[3].append(tdcCounts[2][32:64])#Singlet Eta
            phiHits[3].append(tdcCounts[2][64:128])#Singlet Phi
            etaHits[4].append(tdcCounts[3][0:32])#Double Eta low
            phiHits[4].append(tdcCounts[3][32:96])#Double Phi Low
            etaHits[5].append(tdcCounts[4][96:128])#Doublet Eta top
            phiHits[5].append(tdcCounts[4][0:64])#Doublet Phi top

        else:
            
            print(f"Corrupted Event, {event_corruptions} corrupted events")
            number_corruptions.append(event_corruptions)

        etaCorruption[0]= np.add(etaCorruption[0],corrupted_channels[0][0:32]) #Triplet Eta Low
        phiCorruption[0]= np.add(phiCorruption[0],corrupted_channels[0][32:96]) #Triplet Phi low
        etaCorruption[1]= np.add(etaCorruption[1],corrupted_channels[0][96:128]) #Triplet Eta Mid
        phiCorruption[1]= np.add(phiCorruption[1],corrupted_channels[1][0:64]) #Triplet Phi Mid
        etaCorruption[2]= np.add(etaCorruption[2],corrupted_channels[1][64:96]) #Triplet Eta Top
        phiCorruption[2]= np.add(phiCorruption[2],corrupted_channels[1][96:128]+corrupted_channels[2][0:32]) #Triplet Phi Top
        etaCorruption[3]= np.add(etaCorruption[3],corrupted_channels[2][32:64])#Singlet Eta
        phiCorruption[3]= np.add(phiCorruption[3],corrupted_channels[2][64:128])#Singlet Phi
        etaCorruption[4]= np.add(etaCorruption[4],corrupted_channels[3][0:32])#Double Eta low
        phiCorruption[4]= np.add(phiCorruption[4],corrupted_channels[3][32:96])#Double Phi Low
        etaCorruption[5]= np.add(etaCorruption[5],corrupted_channels[4][96:128])#Doublet Eta top
        phiCorruption[5]= np.add(phiCorruption[5],corrupted_channels[4][0:64])#Doublet Phi top
    

    return etaHits,phiHits,number_corruptions, tdc_corruptions, etaCorruption, phiCorruption

In [32]:
data = ANT.importDatafile(current_directory+"\\ProAnubisData\\60sRun_24_3_4.h5")

etaHits, phiHits, number_corruptions, tdc_corruptions, etaCorruption, phiCorruption = divideHitCountsByRPC_Timed_UNCORRUPTED(data)

Corrupted Event, 4 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 4 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 9 corrupted events
Corrupted Event, 3 corrupted events
Corrupted Event, 3 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 4 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 2 corrupted events
Corrupted Event, 2 corrupted events
Corrupted Event, 5 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 7 corrupted events
Corrupted Event, 5 corrupted events
Corrupted Event, 2 corrupted events
Corrupted Event, 3 corrupted events
Corrupted Event, 4 corrupted events
Corrupted Event, 8 corrupted events
Corrupted Event, 9 corrupted events
Corrupted Event, 2 corrupted events
Corrupted Event, 3 corrupted events
Corrupted Event, 4 corrupted events
Corrupted Event, 1 corrupted events
Corrupted Event, 2 corrupted

IndexError: list index out of range